# LSTM Stock Market Example

Wouldn't you like to predict the stock market?  (Don't try this at home).  Here's an example of how we can use LSTMs to predict patterns in the S&P 500.

In [3]:

from bigdl.nn.layer import *
from bigdl.nn.criterion import * 
from bigdl.optim.optimizer import *
from bigdl.util.common import *
from bigdl.dataset.transformer import *
from bigdl.dataset import mnist
from bigdl.dataset.base import *
from bigdl.util.common import *

import numpy as np
from numpy import newaxis
import lstm, time #helper libraries

In [3]:
learning_rate = 0.1
training_epochs = 100
batch_size = 10
init_engine()

## Loading the Data

We are loading the S&P 500 Data. It is simply a sequence of the s&P 500 per line. For example

```
1095.23
1100.53
1103.74
```

In [8]:
#Step 1 Load Data
X_train, y_train, X_test, y_test = lstm.load_data('sp500.csv', 50, True)

In [9]:
X_train_rdd = sc.parallelize(X_train).map(lambda x: x.flatten())
y_train_rdd = sc.parallelize(y_train)
X_test_rdd = sc.parallelize(X_test).map(lambda x: x.flatten())
y_test_rdd = sc.parallelize(y_test)

In [11]:
x_train_sample = X_train_rdd.zip(y_train_rdd).map(lambda x:Sample.from_ndarray(x[0], x[1]))
X_test_sample = X_test_rdd.zip(y_test_rdd).map(lambda x:Sample.from_ndarray(x[0], x[1]))

# Building the Model

The model is a type of Recurrent Neural Network called an LSTM.

It takes an input of 50 (because we are going to use 50 as the sequence), and an ouptut as 1.

In [12]:
def build_model(input_size, hidden_size, output_size):
    model = Sequential()
    recurrent = Recurrent()
    recurrent.add(LSTM(input_size, hidden_size))
    model.add(InferReshape([-1, input_size], True))
    model.add(recurrent)
    model.add(Select(2, -1))
    model.add(Linear(hidden_size, output_size))
    return model
model = build_model(50, 50, 1)

creating: createSequential
creating: createRecurrent
creating: createLSTM
creating: createInferReshape
creating: createSelect
creating: createLinear


# Training The Model

Notice that the model is a regression, not classificaiton model

In [20]:
# Create an Optimizer
optimizer = Optimizer(
    model=model,
    training_rdd=x_train_sample,
    criterion=MSECriterion(),
    optim_method=SGD(learningrate=learning_rate),
    end_trigger=MaxEpoch(training_epochs),
    batch_size=batch_size)

creating: createMSECriterion
creating: createDefault
creating: createSGD
creating: createMaxEpoch
creating: createOptimizer


# Making Predictions

Here we are going to make predictions based on the model.  We will use sliding windows of length 50.

In [43]:

#Predict sequence of 50 steps before shifting prediction run forward by 50 steps
prediction_seqs = []
data = X_test
prediction_len = 50
window_size=50
for i in range(len(data)//prediction_len):
    curr_frame = data[i*int(prediction_len)]
    predicted = []
    for j in range(int(prediction_len)):

        curr_data = curr_frame[newaxis,:,:]

        curr_data_rdd = sc.parallelize(curr_data).map(lambda x: Sample.from_ndarray(x,[0]))

        preds = model.predict(curr_data_rdd).collect()[0][0][0]
        predicted.append(preds)
        curr_frame = curr_frame[1:]
        curr_frame = np.insert(curr_frame, [window_size-1], predicted[-1], axis=0)
        prediction_seqs.append(predicted)
predictions = prediction_seqs

In [ ]:
# Step 5 -- Plot predictions

lstm.plot_results_multiple(predictions, y_test, 50)